# MAP-Elites using pyribs

In [1]:
import numpy as np
import requests
import json
import random
from ribs.archives import GridArchive
from ribs.emitters import EmitterBase
from ribs.schedulers import Scheduler
from ribs.visualize import grid_archive_heatmap
import matplotlib.pyplot as plt

In [2]:
# Server JS which exposes the trackGenerations and genetic operators 
BASE_URL = 'http://localhost:4242'

## Problem parameters

In [3]:
POINTS_COUNT = 50  # Number of points in dataSet
MAX_SELECTED_CELLS = 10  # Maximum number of selected cells
solution_dim = POINTS_COUNT * 2 + MAX_SELECTED_CELLS * 2  # x and y for each point and selected cell
track_size_range = (2, 5)  
length_range = (400, 3000) # Range of lengths for the track just for the measures in the archive

## Genetic operators

In [4]:
class CustomEmitter(EmitterBase):
    def __init__(self, archive, solution_dim, batch_size=36, bounds=None):
        super().__init__(archive, solution_dim=solution_dim, bounds=bounds)
        self.batch_size = batch_size
        self.iteration = 0

    def ask(self):
        self.iteration += 1
        if self.iteration <= 100:  # Initial population
            return self.generate_initial_solutions()
        elif np.random.random() < 0.5:  # Mutation
            return self.mutate_solutions()
        else:  # Crossover
            return self.crossover_solutions()

    def tell(self, solutions, objectives, measures, add_info, **kwargs):
    
        #archive.add(solutions, measures, objectives)
        pass

    def generate_initial_solutions(self):
        solutions = []
        for _ in range(self.batch_size):
            solution = np.random.uniform(self.lower_bounds, self.upper_bounds, self.solution_dim)
            solutions.append(solution)
        return np.array(solutions)

    def mutate_solutions(self):
        parents = self.archive.sample_elites(self.batch_size)
        mutated = []
        for parent in parents:
            parent_array = parent.solution
            parent_dict = solution_array_to_dict(parent_array, parent.behavior[0])
            response = requests.post(f'{BASE_URL}/mutate', json={
                "individual": parent_dict,
                "intensityMutation": 10  # Adjust as needed
            })
            mutated_dict = response.json()['mutated']
            mutated.append(dict_to_solution_array(mutated_dict))
        return np.array(mutated)

    def crossover_solutions(self):
        offspring = []
        for _ in range(self.batch_size // 2):
            parents = self.archive.sample_elites(2)
            parent1 = solution_array_to_dict(parents[0].solution, parents[0].behavior[0])
            parent2 = solution_array_to_dict(parents[1].solution, parents[1].behavior[0])
            response = requests.post(f'{BASE_URL}/crossover', json={
                "mode": "voronoi",
                "parent1": parent1,
                "parent2": parent2
            })
            child1, child2 = response.json()['offspring']
            offspring.extend([dict_to_solution_array(child1), dict_to_solution_array(child2)])
        return np.array(offspring)

In [5]:
# Create archive, emitter , scheduler

archive = GridArchive(solution_dim=solution_dim,
                      dims=[10, 10],  # 10x10 grid, adjust as needed
                      ranges=[track_size_range, length_range])


emitter = CustomEmitter(archive, 
                        solution_dim=solution_dim, 
                        batch_size=36, 
                        bounds=[(0, 600)] * solution_dim)  # Assuming x and y coordinates are between 0 and 600

scheduler = Scheduler(archive, [emitter])

NameError: name 'length_range' is not defined

### Helper functions

In [6]:
def generate_random_solution():
    return {
        "id": random.random(),
        "mode": "voronoi",
        "trackSize": random.randint(*track_size_range),
        "parents": {
            "parent1": None,
            "parent2": None
        },
        "dataSet": [{"x": random.uniform(0, 600), "y": random.uniform(0, 600)} for _ in range(POINTS_COUNT)],
        "selectedCells": []
    }

# Function to convert solution array to dict
def solution_to_dict(solution):
    dataSet = [{"x": solution[i], "y": solution[i+1]} for i in range(POINTS_COUNT*2)]
    selectedCells = [{"x": solution[i], "y": solution[i+1]} 
                     for i in range(POINTS_COUNT*2, solution_dim, 2) 
                     if solution[i] != 0 or solution[i+1] != 0]  # Ignore (0,0) cells
    return {
        "mode": "voronoi",
        "trackSize": len(selectedCells),
        "dataSet": dataSet,
        "selectedCells": selectedCells
    }

def dict_to_solution_array(solution_dict):
    return np.array([point['x'] for point in solution_dict['dataSet']] +
                    [point['y'] for point in solution_dict['dataSet']])

# Evaluation function
def evaluate_solution(solution):
    solution_dict = solution_to_dict(solution)
    try:
        response = requests.post(f'{BASE_URL}/evaluate', json=solution_dict)
        result = response.json()
        fitness = result['fitness']
        objective = -(fitness['deltaX'] + fitness['deltaY'])
        measures = [result['trackSize'], fitness['length']]
        
        # Update solution with actual selectedCells
        selected_cells = result['selectedCells']
        for i, cell in enumerate(selected_cells):
            if i < MAX_SELECTED_CELLS:
                solution[POINTS_COUNT*2 + i*2] = cell['x']
                solution[POINTS_COUNT*2 + i*2 + 1] = cell['y']
        
        # Fill remaining selectedCells slots with zeros if any
        for i in range(len(selected_cells), MAX_SELECTED_CELLS):
            solution[POINTS_COUNT*2 + i*2] = 0
            solution[POINTS_COUNT*2 + i*2 + 1] = 0
        
        return objective, measures, solution
    except Exception as e:
        print(f"Error in evaluate_solution: {e}")
        return float('-inf'), [-1, -1], solution


## Illuminating search spaces by mapping elites


In [7]:
def run_map_elites(iterations):
    for itr in range(1, iterations + 1):
        # Request solutions from the scheduler
        solutions = scheduler.ask()
        
        # Evaluate the solutions
        objectives = []
        measures_list = []
        for i, solution in enumerate(solutions):
            obj, measures, updated_solution = evaluate_solution(solution)
            obj, measures, new_track_size, selected_cells = evaluate_solution(solution, track_size)
            objectives.append(obj)
            measures_list.append(measures)
            solutions[i] = updated_solution  # Update the solution with actual selectedCells
            
        # Send the results back to the scheduler
        scheduler.tell(objectives, measures_list, solutions)
        
        # Logging
        if itr % 100 == 0:
            print(f"> {itr} itrs completed")
            print(f"  - Archive size: {len(archive)}")
            print(f"  - Archive coverage: {archive.stats.coverage}")

    return archive

In [8]:
final_archive = run_map_elites(10) 

KeyError: 'trackSize'

## Visualize Results

In [ ]:
plt.figure(figsize=(10, 8))
grid_archive_heatmap(final_archive)
plt.title("MAP-Elites Archive")
plt.xlabel("Track Size")
plt.ylabel("Track Length")
plt.show()

## Archive Statistics

In [ ]:
print(f"Total solutions: {len(final_archive)}")
print(f"Best solution: Objective = {final_archive.stats.obj_max}")
best_solution = final_archive.elite_with_behavior(final_archive.best)
print(f"Best solution measures: {best_solution.behavior}")

best_track = solution_array_to_dict(best_solution.solution, best_solution.behavior[0])
print("Best track data:", json.dumps(best_track, indent=2))